In [ ]:
from typing import Dict, List, Tuple
import sqlite3
from pathlib import Path

import requests
from IPython.display import display, HTML
from bs4 import BeautifulSoup

In [ ]:
def get_url(url: str) -> str:
    try:
        response = requests.get(url)
        if response.ok:
            return response.content.decode('utf-8')
        else:
            raise Exception('invalid response code', response)
    except Exception as e:
        raise e

In [ ]:
main_html = get_url('http://www.dailymail.co.uk/news/worldnews/index.html')
soup = BeautifulSoup(main_html, 'lxml')

In [ ]:
articles = []

articles = soup.find_all('div', {'class': 'article'})
# articles = [a for i, a in enumerate(articles) if i in [0, 1, 3]]
articles = [sc.find_all('h2') for sc in articles]
articles = [a.find('a')
           for art in articles
           for a in art]


headlines = [a.text for a in articles]
print(len(headlines))
print(headlines)

In [ ]:
base_url = 'http://www.dailymail.co.uk'
article_urls = [base_url + a.get('href') for a in articles]
article_urls

In [ ]:
def extract_image(url):
    print('.', end='')
#     print(url)
    main_html = get_url(url)
    soup = BeautifulSoup(main_html, 'lxml')
    soup = soup.find('div', {'id': 'js-article-text'})
    
    image_url = 'http://i.dailymail.co.uk/i/pix/2016/04/06/02/0BFCB585000005DC-3525479-image-m-25_1459906645394.jpg'
    img = soup.find('img')
    if img is not None:
        image_url = img.get('data-src', '')
    
    excerpt = ''
    ul = soup.find('ul', {'class': 'mol-bullets-with-font'})
    if ul is not None:
        tmp = ul.find_all('li')
        if tmp is not None:
            tmp = [t.text for t in tmp]
#             tmp = [t.find(['strong', 'b', 'p']) for t in tmp]
#             if tmp is not None:
#                 tmp = [t.text for t in tmp]
            excerpt = ' '.join(tmp)
    
    text = ''
    body = soup.find('div', {'itemprop': 'articleBody'})
    if body is not None:
        text = body.find_all('p', recursive=False)
        text = [t.text for t in text]
        text = '\n'.join(text)
    return image_url, text, excerpt

# extract_image(article_urls[-5])
tmp = [extract_image(url) for url in article_urls]
image_urls, texts, excerpts = list(zip(*tmp))

In [ ]:
image_urls

In [ ]:
excerpts

In [ ]:
len(texts)

In [ ]:
print(texts[0])

In [ ]:
path_data = Path('sql')
path_data.mkdir(exist_ok=True)

db_file = path_data / 'db.sqlite'
create_tables_file = path_data / 'create_tables.sql'


company_name = 'dailymail'

with sqlite3.connect(str(db_file)) as con:
    cur = con.cursor()
    cur.execute("select count(*) from company")
    print(cur.fetchone()[0])
    cur.execute("select count(*) from article")
    print(cur.fetchone()[0])
    
    cur.execute(f'SELECT source_id from company WHERE name = ?', (company_name,))
    x = cur.fetchone()
    if x and len(x) > 0:
        cur.execute(f'DELETE FROM article WHERE source_id = ?', (x[0],))
        
    cur.execute(f'DELETE FROM company WHERE name = ?', (company_name,))
    cur.execute(f'INSERT INTO company (name) VALUES (?)', (company_name,))
    cur.execute(f'SELECT source_id from company WHERE name = ?', (company_name,))
    x = cur.fetchone()
    if x and len(x) > 0:
        company_id = x[0]
    else:
        raise Exception('?')
    con.commit()
        
    cur.execute("select count(*) from article")
    print(cur.fetchone()[0])
    for h, e, i, a, t in zip(headlines, excerpts, image_urls, article_urls, texts):
        cur.execute(f'INSERT INTO article (source_id, headline, excerpt, image_url, article_url, full_text) VALUES (?, ?, ?, ?, ?, ?)',
                    (company_id, h, e, i, a, t))
    con.commit()    
    cur.execute("select count(*) from article")
    print(cur.fetchone()[0])
    
    cur.execute("select count(*) from company")
    print(cur.fetchone()[0])